In [1]:
import time
import cv2
from cv2 import VideoCapture, imwrite
import os
import datetime

prev_gray = None

run = True

motion_dir = None

sleep_tm = 1

cam = VideoCapture(0)

while run:
    
    result, image = cam.read() 

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #gray = cv2.GaussianBlur(gray, (21, 21), 0)

    hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()

    if prev_gray is not None:
        d = cv2.compareHist(hist, prev_gray, cv2.HISTCMP_CORREL)
        print(d)
        if d < 0.95:
            sleep_tm = 1
            ts_now = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

            if motion_dir is None:
                motion_dir = f"motions/motion_{ts_now}"
                os.makedirs(motion_dir)
            imwrite(f"{motion_dir}/event_{ts_now}.png", image)  
            print("Motion detected set: ", motion_dir)              
                
        else:
            motion_dir = None
            print("No motion detected")
            sleep_tm = 2

    prev_gray = hist

    print("Sleeping for 3 seconds")
    time.sleep(sleep_tm)

In [ ]:
!